## Data Transformation for Gold Lakehouse

In [ ]:
# Load data to the dataframe and create the gold layer for Csutomer table

df = spark.sql("SELECT CustomerID as CustomerKey,FirstName,LastName ,FullName ,DateInserted FROM Sales_Silver.Customers")

# Saving the customer data to the Delta Lake path for DimCustomer in Sales_Gold Lakehouse
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimCustomer")

# Display the first 10 rows of the dataframe to preview your data
display(df.head(10))

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cdf194fd-e3ee-4f02-808b-0c05da8507ef)

#### Creating the_Employee Gold dimension tables

In [ ]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW tmpEmployee
AS
SELECT   EmployeeID EmployeeKey
            ,ManagerID
            ,FirstName
            ,LastName
            ,FullName
            ,JobTitle
            ,OrganizationLevel
            ,MaritalStatus
            ,Gender
            ,Territory
            ,Country
            ,Group
FROM Sales_Silver.Employees

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
df = spark.sql("SELECT * FROM tmpEmployee")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimEmployee")

# Display the first 10 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 5, Finished, Available, Finished)

#### Creating the Products_Gold dimension tables

In [ ]:
%%sql
--nb_DimProduct_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProducts
AS
SELECT    ProductID as ProductKey
            ,ProductNumber
            ,ProductName
            ,ModelName
            ,MakeFlag
            ,StandardCost
            ,ListPrice
            ,SubCategoryID
            ,DateInserted
FROM Sales_Silver.Products    

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
df = spark.sql("SELECT * FROM tmpProducts")

df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProduct")

# Display the first 5 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 7, Finished, Available, Finished)

#### Creating the ProductsCategories_Gold dimension tables

In [ ]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW tmpProductCategory
AS
SELECT       CategoryID as CategoryKey
            ,CategoryName
            ,DateInserted
FROM Sales_Silver.ProductCategories 

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
df = spark.sql("SELECT * FROM tmpProductCategory")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProductCategory")

# Display the first 5 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 9, Finished, Available, Finished)

#### Creating the ProductsSubCategories_Gold dimension tables

In [ ]:
%%sql
--nb_DimProductSubCategories_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProductSubCategory
AS
SELECT      SubCategoryID as SubCategoryKey
            ,CategoryID as CategoryKey
            ,SubCategoryName
FROM Sales_Silver.ProductSubCategories

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
df = spark.sql("SELECT * FROM tmpProductSubCategory")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProductSubCategory")

# Display the first 5 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 11, Finished, Available, Finished)

#### Creating the FactSales_Gold dimension tables

In [ ]:
%%sql
--nb_FactSales_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpSales
AS
SELECT      SalesOrderID AS SalesOrderKey
            ,ifnull(e.EmployeeKey,-1) as EmployeeKey
            ,ifnull(c.CustomerKey,-1) as CustomerKey
            ,ifnull(p.ProductKey,-1) as ProductKey
            ,ifnull(pc.CategoryKey,-1) as CategoryKey
            ,ifnull(psc.SubCategoryKey,-1) as SubCategoryKey                       
            ,SalesOrderDetailID
            ,OrderDate
            ,DueDate
            ,ShipDate
            ,SubTotal
            ,TaxAmt
            ,Freight
            ,TotalDue
            ,OrderQty
            ,UnitPrice
            ,UnitPriceDiscount
            ,LineTotal
            ,o.DateInserted
FROM Sales_Silver.Orders o
LEFT JOIN Sales_Gold.DimCustomer c
ON c.CustomerKey =o.CustomerID

LEFT JOIN Sales_Gold.DimEmployee e 
ON e.EmployeeKey= o.EmployeeID

LEFT JOIN Sales_Gold.DimProduct p 
ON p.ProductKey = o.ProductID

LEFT JOIN Sales_Gold.DimProductSubCategory psc
ON psc.SubCategoryKey=p.SubCategoryID

LEFT JOIN Sales_Gold.DimProductCategory pc
ON pc.CategoryKey= psc.CategoryKey

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 12, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [ ]:
df = spark.sql("SELECT * FROM tmpSales")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/FactSales")

# Display the first 10 rows of the dataframe to preview your data
display(df.head(10))

StatementMeta(, f2ff21a1-29ed-4b05-a26e-536ca34440a2, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 217aad7b-5d5e-4161-aec1-2d495e29c6f9)